### REQURIED LIB


In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
# Check if CUDA (GPU) is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

## Data Preprocessing and Loading Dataset

In [ ]:
# transformations for the images
transform = transforms.ToTensor()

# Load the MNIST dataset and create data loaders
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = MNIST(root='./data', train=False, transform=transform, download=True)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [ ]:
train_dataset

## Visualizing the Data
Let's visualize a few examples from the dataset to understand the input images better.

In [ ]:
train_dataset[0][0].shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to display a grid of images
def show_images(images, labels, nrows, ncols):
    fig, axes = plt.subplots(nrows, ncols, figsize=(10, 6))
    for i, ax in enumerate(axes.flat):
        ax.imshow(np.squeeze(images[i]), cmap='gray')
        ax.set_title(f"Label: {labels[i]}")
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# Get a batch of images and labels from the test dataset
images, labels = next(iter(test_dataloader))

# Display the images and their labels
show_images(images, labels, 4, 8)

In [ ]:
img , label = train_dataset[0]
plt.imshow(np.squeeze(img), cmap='gray')

## Model Architecture
Next, we'll define our CNN model architecture using PyTorch.


In [ ]:
#CNN model architecture using PyTorch framework
class DigitClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3)
        self.L1 = nn.Linear(in_features=8*5*5, out_features=64)
        self.L2 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):
        x = self.conv1(x)  # 1*28*28 --> 16*24*24
        x = self.pool(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.pool(x)
        x = F.relu(x)

        x = x.flatten(start_dim=1, end_dim=-1)
        x = F.relu(self.L1(x))
        x = F.relu(self.L2(x))
        return x

#Training the Model


Define Loss Function and Optimizer
We need to specify the loss function and optimizer for training the model.

In [ ]:
# Instantiate the model and move it to the appropriate device
model = DigitClassification().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01,)

## Training Loop
Now, we'll implement the training loop to train our model on the MNIST dataset.

In [ ]:
def train(model, train_dataloader, criterion, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in train_dataloader:
        data, label = batch[0].to(device), batch[1].to(device)  # Move data to device

        optimizer.zero_grad()

        predicted = model(data)
        loss = criterion(predicted, label)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calculate accuracy
        _, predicted_labels = torch.max(predicted, 1)
        correct += (predicted_labels == label).sum().item()
        total += label.size(0)

    accuracy = correct / total
    return total_loss / len(train_dataloader), accuracy

### TEST loop


In [ ]:
def test(model, test_dataloader, criterion, optimizer):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # No need to track gradients during testing
        for batch in test_dataloader:
            data, label = batch[0].to(device), batch[1].to(device)  # Move data to device

            predicted = model(data)
            loss = criterion(predicted, label)

            total_loss += loss.item()

            # Calculate accuracy
            _, predicted_labels = torch.max(predicted, 1)
            correct += (predicted_labels == label).sum().item()
            total += label.size(0)

    accuracy = correct / total
    return total_loss / len(test_dataloader), accuracy

In [ ]:
# Train and test the model for a certain number of epochs
num_epochs = 30
training_losses = []
testing_losses = []
training_accuracies = []
testing_accuracies = []

In [ ]:
for epoch in tqdm(range(num_epochs), desc='Training Progress'):

    training_loss, training_accuracy = train(model, train_dataloader, criterion, optimizer)
    testing_loss, testing_accuracy = test(model, test_dataloader, criterion, optimizer)

    # Append losses and accuracies for visualization
    training_losses.append(training_loss)
    testing_losses.append(testing_loss)
    training_accuracies.append(training_accuracy)
    testing_accuracies.append(testing_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}: Training Loss: {training_loss}, Testing Loss: {testing_loss}, Training Accuracy: {training_accuracy}, Testing Accuracy: {testing_accuracy}")

In [ ]:
# Plotting training and testing losses
plt.plot(training_losses, label='Training Loss')
plt.plot(testing_losses, label='Testing Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Testing Loss')
plt.legend()
plt.show()

In [ ]:
# Plot accuracies
plt.figure(figsize=(10, 5))
plt.plot(training_accuracies, label='Training Accuracy')
plt.plot(testing_accuracies, label='Testing Accuracy')
plt.title('Training and Testing Accuracies')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Predictions and Visualization
## Making Predictions
Let's make predictions on new data using the trained model.

In [ ]:
# Function to make predictions on new data
def predict(model, dataloader):
    predictions = []
    with torch.no_grad():
        for data in dataloader:
            inputs = data[0].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.cpu().numpy())
    return predictions

In [ ]:
# Make predictions on the test dataset
test_predictions = predict(model, test_dataloader)

In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models

def process_input_image(image_path):
    # Load the image and convert to grayscale
    img = Image.open(image_path).convert('L')
    # Resize to 28x28 and convert to numpy array
    img = img.resize((28, 28))
    img_array = np.array(img)
    # Normalize pixel values
    img_array = img_array.astype('float32') / 255.0
    # Add a batch dimension and return
    return np.expand_dims(img_array, axis=0)

## Visualizing Predictions
We'll visualize a random sample of test images along with their predicted labels.

In [ ]:
# Function to plot a random sample of images along with their predicted labels
def visualize_predictions(dataset, predictions, num_samples=5):
    plt.figure(figsize=(15, 7))
    samples = np.random.choice(len(dataset), num_samples, replace=False)
    for i, idx in enumerate(samples):
        plt.subplot(1, num_samples, i + 1)
        image, label = dataset[idx]
        plt.imshow(image.squeeze(), cmap='gray')
        plt.title(f"Predicted: {predictions[idx]}, Actual: {label}")
        plt.axis('off')
    plt.show()


In [ ]:
# Visualize predictions on a random sample of test images
visualize_predictions(test_dataset, test_predictions)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Reshape the data
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

# Define the model
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(8, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

def visualize_predictions(dataset, predictions, num_samples=5):
    plt.figure(figsize=(15, 7))
    samples = np.random.choice(len(dataset), num_samples, replace=False)
    for i, idx in enumerate(samples):
        plt.subplot(1, num_samples, i + 1)
        image, label = dataset[idx]
        plt.imshow(image.squeeze(), cmap='gray')
        plt.title(f"Predicted: {predictions[idx]}, Actual: {label}")
        plt.axis('off')
    plt.show()
